# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure the gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load 
file_to_load = "../output_data/cities_weather.csv"

# Read File and store into Pandas data frame
cities_df = pd.read_csv(file_to_load)


In [3]:
cities_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nabire,-3.3667,135.4833,76.41,92,100,3.31,ID,1631625356
1,1,Capaci,38.1711,13.2393,90.52,62,20,1.99,IT,1631625357
2,2,San,13.3034,-4.8956,94.91,42,19,6.46,ML,1631625359
3,3,Pascagoula,30.3658,-88.5561,77.05,96,40,5.75,US,1631625360
4,4,Lompoc,34.6391,-120.4579,59.09,99,90,0.00,US,1631625361


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = cities_df[["Latitude","Longitude"]]
humidity = cities_df["Humidity"].astype(float)

In [5]:
max(humidity)

100.0

In [6]:
# Plot Heatmap

fig_layout = {"width": "100%",
             "height": "600px",
              "padding":"1px"}

fig = gmaps.figure(layout = fig_layout, center =(20.0,10.0), zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 max_intensity = max(humidity),
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
cities_df.shape

(540, 10)

In [8]:
#A max temperature lower than 80 degrees but higher than 65.
weather_conditions_df = cities_df[(cities_df["Max Temp"] >65) & (cities_df["Max Temp"] <= 80)]

weather_conditions_df.shape

(178, 10)

In [9]:
weather_conditions_df = weather_conditions_df[(weather_conditions_df["Wind Speed"] < 11)]
weather_conditions_df.shape

(140, 10)

In [10]:
# Zero cloudiness.
weather_conditions_df = weather_conditions_df[(weather_conditions_df["Cloudiness"] == 0)]
weather_conditions_df.shape

(10, 10)

In [67]:
# #A max temperature lower than 80 degrees but higher than 65.
# weather_conditions_df = cities_df[(cities_df["Max Temp"] > 65) & (cities_df["Max Temp"] <= 80)]

# # Wind speed less than 11 mph.
# weather_conditions_df = weather_conditions_df[(weather_conditions_df["Wind Speed"] < 11)]

# # Zero cloudiness.
# weather_conditions_df = weather_conditions_df[(weather_conditions_df["Cloudiness"] == 0)]

In [11]:
#ideal weather condition data
weather_conditions_df.shape

(10, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = weather_conditions_df

In [13]:
#Reset the index
hotel_df.reset_index(inplace=True)
hotel_df.drop("index",axis=1,inplace=True)
hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,261,Nur-Sultan,51.1801,71.4460,66.15,34,0,8.95,KZ,1631625488
1,274,Yulara,-25.2406,130.9889,73.24,21,0,3.44,AU,1631625668
2,295,Kumul,42.8000,93.4500,78.64,19,0,8.34,CN,1631625684
3,301,Havelock,34.8791,-76.9013,79.95,92,0,1.99,US,1631625689
4,415,Enshi,30.3000,109.4833,72.10,82,0,1.57,CN,1631625520


In [14]:
hotel_df['Hotel Name'] = ""
for i, row in hotel_df.iterrows():
#for i in hotel_df:
    try:
        #Build URL using the Google Maps AP1
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        #write params dict
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_response = requests.get(base_url, params=params).json()
        
        hotel_df.loc[i, "Hotel Name"] = hotel_response["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[i, "Hotel Name"] = "NaN"
        

In [19]:
hotel_df.head(20)

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,261,Nur-Sultan,51.1801,71.4460,66.15,34,0,8.95,KZ,1631625488,"Radisson Hotel, Astana"
1,274,Yulara,-25.2406,130.9889,73.24,21,0,3.44,AU,1631625668,Sails in the Desert
2,295,Kumul,42.8000,93.4500,78.64,19,0,8.34,CN,1631625684,哈密环球大酒店
3,301,Havelock,34.8791,-76.9013,79.95,92,0,1.99,US,1631625689,Holiday Inn Express & Suites Havelock NW-New B...
4,415,Enshi,30.3000,109.4833,72.10,82,0,1.57,CN,1631625520,Moevenpick Enshi
5,450,Bois-Guillaume,49.4602,1.1222,74.98,73,0,10.36,FR,1631625842,ibis budget Rouen Nord Isneauville
6,465,Seoul,37.5683,126.9778,74.77,57,0,8.05,KR,1631625703,Nine Tree Premier Hotel Myeongdong II
7,486,Skwierzyna,52.5991,15.5065,71.67,47,0,0.58,PL,1631625878,Dom nad Rzeką
8,520,Korla,41.7597,86.1469,77.32,22,0,7.43,CN,1631625913,The Garden Hotel Korla
9,522,Along,28.1667,94.7667,79.74,78,0,0.54,IN,1631625913,Hotel West


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Plot Heatmap
humidity=hotel_df["Humidity"].astype(float)
fig_layout = {"width": "100%",
             "height": "600px",
              "padding":"1px"}

fig = gmaps.figure(layout = fig_layout, center =(20.0,10.0), zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,
                                 max_intensity = max(humidity),
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='600px', padding='1px', width='100%'))